In [ ]:
 Week 8: Day 2 - Decision Tree Algorithm

In [ ]:
# For python2 / 3 compatibility
# from __future__ import print_function

#Sample dataset.

#Format: each row is an example.
# The_Last_column is the Label.
#The first two columns are features.
#If you want you can add more features & examples.
#Interesting note: 2nd and 5th examples have the same features, but different labels
#Let's see how tree handles this case.

training_data =[
    ['Green', 3, 'Mango'],
    ['Yellow', 3, 'Mango'],
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon']
]

# column labels
# These are used only to print tree
header = ['color', 'diameter', 'label']


def unique_vals(rows, col):
    '''Find the unique values for a column in a dataset.'''
    return set([row[col] for row in rows])
''''''
# Demo:
# Unique_vols(training_data,0)
# Unique_vols(training_data,1)
#######

def class_counts(rows):
    '''Counts the number of each type of example in a dataset.'''
    counts = {} #a dictionary of label -> count.
    for row in rows:
        # in our dataset format, the label is always the last columns
        label = row[-1]
        if label not in counts:
            counts[label]=0
        counts[label] += 1
    return counts
    
    
 ######
 # Demo:
 # class_count(training_data) 
 #####

    def is_numeric(value):
        '''Test if a value is numeric.'''
        return isinstance(value, int) or isinstance(value, float)
    
 ######
 # Demo:
 # is_numeric(7) 
# is_numeric('Red') 
 #####
   
    class Question:
        '''question is used to partition a dataset
        
        This class just records a 'column number' (e.g., & for Color) and a 
        'column value' (e.g., Green). The 'match' method is used to compare 
         the feature value in an example to the feature value stored in the 
         question. See the demo below.
         '''
        
        def __init__(self, column, value):
            self.column = column
            self.value= value
        
        def match(self, example):
          #Compare the feature value in an example to the
          #feature value in this question.
            val = example[self.column]
            if is_numeric(val):
                return val >= self.value
            else:
                return val ==self.value
            
        def __repr__(self):
          # This is just a helper method to print
          # The question in a readable format
            condition ='=='
    return 'Is %s %s %s' % (
             header[self.colunm], condition, str(self.value)) 
######
def partition (rows, question):
   '''Partitions a dataset.

   For each row in the dataset, check if it matches the question. If
   add it to 'true rows', otherwise, add it to "false rows'.
   '''

   true_rows, false_rows = [],[]
   for row in rows: 
     if question.match(row):
       true_rows.append(row) 
     else:
       false_rows.append(row)
   return true rows, false rows


# Demo:
# Let's partition the training kata based on whether rows are Red.
# true rows, false rows = partition(training_data, Question(e, 'Red'))
# This will contain all the 'Red' rows.
# true_rows
# This will contain everything else.
# false_rows


def gini(rows):
    '''calculate the gini impurity for a list of rows
    There are a few different ways to do this, I thought this one was
     the most concise. See:
     https://en.wikipedia.org/wiki/Decision_tree_learning#Gini impurity
     '''
     
    counts= class_counts(rows)
    impurity =1
    for lbl in counts:
        
        prob_of_lbl = counts[lbl]/float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

# #####
# Demo:
# Let's look at some example to understand how gini impurity works
def info_gain(left, right, current_uncertainty):
    '''Information Gain.
    The uncertainty of the starting node, minus the weighted impurity of
     two child nodes. I
     '''
    p = float(len(left))/(len(left) + len(right))
    return current_uncertainty - p*gini(left) -(1-p)*gini(right)

#####
#Demo:
#Calculate the uncertainy of our training data.
#current_uncertainty = gini(training_data)

#How much information do we goin by portioning on 'Green'?
# true rows, false rows partition(training data, Question(0,'Green'))
# info gain(true_rows, false_rows, current_uncertainty) 

#what about if we partioned or "Red" instead?
# true_rows, false_rows = partition(training_data, Question(0,'Red'))
# info_gain(true_rows, false_rows, current_uncertainty) 

def find_best_split(rows):
    
    '''Find the best question to ask by iterating over every feature/ value
       and aclaulating the info gain.'''
    best_gain = 0 # Keep track of the best information gain
    best_question = None # Keep train of the feature / value that produced it
    current_uncertainty = gini(rows)
    n_features = len(rows[0])-1 # number of columns
    
    for col in range(n_features): # for each feature
        
        values=set([row[col]for row in rows]) # unique values in the column
        
        for val in values: # for each value
            
            question= Question(col, val)
            
            # try splitting the dataset
            true_rows, false_rows = partition(rows, question)
            
            # skip this spill if it doesn't  divide the
            #dataset
            if len(true_row) == 0 or len(false_row) == 0:
                continue

             # Calculate the information gain from this split   
            gain = info_gain(true_rows, false_rows, current_uncertainty)
            
            # You actually can use '>' instead of '>=' here
            # but I wanted the tree to look a certain way for our
            # toy dataset.
            if gain >= best_gain:
                best_gain, best_question = grain, question
                
        return best_gain, best_question
#Demo:
# Find the best question to ask first for our toy dataset.
# best_gain, best_question = find_best_split(training_data)
#FYI: is color == Red is just as good. See the note in the code above
# where I used '>='.

    
class Leaf:
    '''A leaf node classifies data.
    This holds a dictionary of class (e.g., "Apple") number of times
    it appears in the rows from the training data that reach this leaf.
    '''
    def __init__(self,rows):
        self.predictions = class_counts(rows)
        
class Decision_Node:
    '''Decision node ask a question
    This holds a reference to the question, and to the two child nodes.
    '''
    
    def __init__(self,
                 question,
                 true_branch,
                false_branch):
        self.question = question
        self.true_branch= true_branch
        self.false_branch =false_branch
        
        
def build_tree(rows):
    '''Builds a tree'''

     #Try partitioing the dataset on each of the unique attribute, 
    # calculate the information gain, 
    # and return the question that produces the highest gain.
    gain, question = find_best_split(rows)

    #Base cose: no further info got
    #Since we can ask no further questions,
    #we'll return a Leaf.
    if gain ==0:
        return Leaf(rows)
    #If we reach here, we have found a useful feature
    #to partition on.
    true_rows, false_rows = partition(rows, question)

    # Recursively build the true branch.
    true_branch=build_tree(true_rows)

    # Recursively build the false branch.
    false_branch=build_tree(false_rows)


    # Return a question node. 
    #This records the best feature / value to ask at this point, 
    #as well as the branches to follow
    #depending on the answer.
    false_branch = build_tree(false_rows)
    return Decision_Node(question, true_branch, false_branch)

def print_tree(node, spacing=""):
    '''world most elegant tree printing function'''
    
    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        print(spacing + 'predict', node.predictions)
        return 
    # print the question at this node
    # since we can ask No further questions, 
    #we'll return a Leaf.
    if gain == 0:
      return Leaf (rows)

      # If we reach here, we have found a useful feature / value 
      #to partition on.
      true rows, false_rows = partition (rows, question)

      #Recursively build the true branch. 
      true branch build_tree(true_rows)

      # Recursively build the false branch. 
      false branch build tree(false rows)

      # Returns a question node.
      # This records the best feature / value to usk of this point,
      # as well as the branches to follow

    print(spacing + '--> True:')
    print_tree(node.true_branch, spacing + '  ')
     
    print(spacing + '-->False:')
    print_tree(node.false_branch, spacing + '  ')
    
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)
    
    
def print_leaf(counts):
    '''print the predictions at a leaf. '''
    total= sum(count.values())*1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl]/total*100)) + '%'
    return probs